In [1]:
from mittens import GloVe
from mittens import Mittens
import pandas as pd
import numpy as np
import nltk
import csv
import re
import operator

/Users/timaiken/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_json('data.json')
num_reviews, review_length = data.shape

In [5]:
unknown_token = '<UNK>'

In [6]:
vocab = {}
for r in range(num_reviews):
    for w in range(review_length):
        
        currWordStruct = data[w][r]
        if currWordStruct == None:
            break
        
        currWord = currWordStruct[0]
        
        if currWord in vocab:
            vocab[currWord] += 1
        else:
            vocab[currWord] = 1

sort = sorted(vocab.items(), key=operator.itemgetter(1), reverse=True)[:4999]
top_5k = {}
counter = 0
for word, _ in sort:
    top_5k[word] = counter
    counter += 1

top_5k[unknown_token] = counter

np.save('5k_vocab_dict.npy', top_5k) 

In [7]:
co_dict = {}
for r in range(num_reviews):
    for w in range(review_length):
        
        currWordStruct = data[w][r]
        if not currWordStruct:
            continue
            
        currWord = currWordStruct[0]

        if currWord not in top_5k:
            currWord = unknown_token

        for other_w in range(-5, 6):

            if ((w + other_w) < 0) or ((w + other_w) >= review_length) or (other_w == 0):
                continue

            otherWordStruct = data[w + other_w][r]
            
            if not otherWordStruct:
                continue
            
            otherWord = otherWordStruct[0]
            
            if otherWord not in top_5k:
                otherWord = unknown_token
            
            dist_weight = 1. / abs(other_w)

            if other_w < 0:
                if (otherWord, currWord) in co_dict:
                    co_dict[(otherWord, currWord)] += dist_weight
                else:
                    co_dict[(otherWord, currWord)] = dist_weight

            else:
                if (currWord, otherWord) in co_dict:
                    co_dict[(currWord, otherWord)] += dist_weight
                else:
                    co_dict[(currWord, otherWord)] = dist_weight

In [8]:
co_matrix = np.zeros((5000, 5000))
for word1, word2 in co_dict.keys():
    co_matrix[top_5k[word1], top_5k[word2]] = co_dict[(word1, word2)]

In [9]:
def glove2dict(glove_filename):
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

original_embeddings = glove2dict("glove.6B.300d.txt")

In [11]:
vocab_array = vocab.keys()
mittens_model = Mittens(n=300, max_iter=500)
new_embeddings = mittens_model.fit(co_matrix, vocab = top_5k.keys(), initial_embedding_dict = original_embeddings)

np.save('GloVe_codeswitch_5k.npy', new_embeddings)

Iteration 500: loss: 159.19223022460938

In [19]:
embeddings = np.load('GloVe_codeswitch_5k.npy')
vocab = np.load("5k_vocab_dict.npy").item()
inv_vocab = {v: k for k, v in vocab.items()}

word = 'have' # pero, entonces, porque, don't, have

idx = vocab[word]
encoding = embeddings[idx,:]

dists = np.sum(np.square(embeddings - encoding), axis=1)

most_related = np.argpartition(dists, 10)[:10]

print("Original word:", word)
for val in most_related:
    if val == idx: continue
    print("Related word: ", inv_vocab[val])

print(most_related)

Original word: have
Related word:  having
Related word:  had
Related word:  been
Related word:  say
Related word:  already
Related word:  come
Related word:  both
Related word:  should
Related word:  yet
[555 113 190 181 335  33 226 877 300 814]
